In [29]:
import requests
from bs4 import BeautifulSoup as bs
import sqlite3
import json
import pandas as pd

In [30]:
url1 = "https://hh.ru/search/vacancy?text=middle+python+developer&area=113&items_on_page=100&search_field=name"
url2 = "https://api.hh.ru/vacancies?text=middle+python+developer&area=113&per_page=100&search_field=name" 
headers = {'User-agent': 'Mozilla/5.0'}


In [31]:
#загрузка данные по вакансиям из web

    search_result = requests.get(url1, headers=headers)
    if search_result.status_code == 200:
        soup = bs(search_result.content, 'lxml')
        list_ = []
        result = soup.find('template').text
        links = json.loads(result)
        for link in links['vacancySearchResult']['vacancies']:   
            vacancy = requests.get(link['links']['desktop'], headers=headers)
            if vacancy.status_code == 200:
                content = bs(vacancy.content, 'lxml')
                description = ''
                vd = content.find('div', attrs={'data-qa': 'vacancy-description'})
                if vd is not None:
                    description = vd.get_text()
                key_skills_containers = content.find_all('span', attrs={'data-qa': 'bloko-tag__text'})
                key_skills =[]
                for skill in key_skills_containers:
                    key_skills.append(skill.text)
                key = ', '.join(key_skills)
                try:
                    company = link['company']['name']
                    list_.append([company, link['name'], description, key])
                except KeyError:
                    #continue
                    company  = 'no'
                    list_.append([company, link['name'], description, key])
    else:
        print(f'Не удалось подключиться: {search_result.status_code}')


    try:
        conn = sqlite3.connect('hw1.db')
        cur = conn.cursor()
        cur.execute("CREATE TABLE web (company text, name text, description text, key_skills text);")
        cur.executemany("INSERT INTO web VALUES(?, ?, ?, ?);", list_)
        conn.commit()
        print('Таблица загружена')
    except sql.Error as error:
        print(f"Не удалось вставить данные в таблицу")
        print("Исключение: ", error.__class__, error.args)    
    
    conn.close()
    print("Соединение закрыто")

Таблица загружена
Соединение закрыто


In [32]:
#Загрузка через API

result = requests.get(url2)
if result.status_code == 200:
    vacancies = result.json().get('items')
    df = []
    for i, vacancy in enumerate(vacancies):
        url = vacancy['url']
        result = requests.get(url)
        vacancy = result.json()
        key = []
        description = ''
        if vacancy['key_skills'] != None:
            key_skills = vacancy['key_skills']
            if key_skills:
                for skill in key_skills:
                   key.append(skill['name'])
            else:
                key.append('нет')
        key = ', '.join(key)
        if vacancy['description'] != None:
            description = vacancy['description']   
        dict_ = {'company': vacancy['employer']['name'], 'name': vacancy['name'],
                 'description': description, 'key_skills': key}
        df.append(dict_)
    df = pd.DataFrame(df)
else:
    print(f'Не удалось подключиться: {search_result.status_code}')
    
try:
    conn = sqlite3.connect('hw1.db')
    cur = conn.cursor()
    cur.execute("CREATE TABLE API (company text, name text, description text, key_skills text);")
    df.to_sql('API', conn, if_exists='append', index = bool)
    conn.commit()
    print('Таблица загружена')
except sql.Error as error:
    print(f"Не удалось вставить данные в таблицу")
    print("Исключение: ", error.__class__, error.args)    
    
conn.close()
print("Соединение закрыто")

Таблица загружена
Соединение закрыто


In [ ]:
#с асинхронной загрузкой пока не смог разобраться